In [1]:
import argparse
import csv
import numpy as np
import os
import pandas as pd
from pprint import pprint
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

gpt35_df = pd.read_csv('../conditional/data/112_gsm8k_gpt35_cot_onesent_responses.csv')

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="auto")
model.generation_config = GenerationConfig.from_pretrained(model_name)
model.generation_config.pad_token_id = model.generation_config.eos_token_id

/Users/vivekvajipey/miniconda3/envs/nightly/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:28<00:00, 14.08s/it]


Get All Transitions

In [5]:
gsm_questions = gpt35_df['Question'].tolist()
reading_question = gsm_questions[3]
reading_question

'Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?'

In [9]:
from collections import Counter, defaultdict
all_equivalence_classes = {"reading_question": Counter([(reading_question,)])}
all_equivalence_classes

{'reading_question': Counter({('Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?',): 1})}

In [10]:
new_equivalence_classes = list(all_equivalence_classes.keys())
new_equivalence_classes

['reading_question']

In [11]:
unique_wordings = defaultdict(set)
unique_wordings["reading_question"] = {reading_question}
unique_wordings

defaultdict(set,
            {'reading_question': {'Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?'}})

In [12]:
prev_equivalence_classes = new_equivalence_classes
prev_equivalence_classes

['reading_question']

In [14]:
class_name = prev_equivalence_classes[0]
class_name

'reading_question'

In [15]:
all_equivalence_classes[class_name]

Counter({('Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?',): 1})

In [16]:
import random

def sample_paths(path_counter, n_samples):
    """
    Sample from a path counter
    """
    return random.choices(
        list(path_counter.keys()), weights=list(path_counter.values()), k=n_samples
    )

In [18]:
n_samples = 2

paths = sample_paths(all_equivalence_classes[class_name], n_samples)
paths

[('Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?',),
 ('Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?',)]

In [22]:
path = paths[0]
path

('Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?',)

<model.sample>

In [23]:
# SAMPLING COMPLETION FROM MODEL
# convert path to tokens, use stopping criteria etc. to generate new path
from utils import get_prompt_message

# NEED TO JOIN COMPONENTS IF COMPLETING PARTIAL TRACE...
question_vector = tokenizer.apply_chat_template(get_prompt_message(path[0], 0), add_generation_prompt=True, return_tensors="pt")
input_tensor = question_vector.repeat(n_samples, 1)
input_tensor.shape

PROMPT:  Q: Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?
A: Let's think step by step.


torch.Size([2, 76])

In [41]:
from transformers import StoppingCriteria, StoppingCriteriaList

class BatchSentenceStoppingCriteria(StoppingCriteria):
    def __init__(self, tokenizer, stop_sequences=None):
        # Encode each stop sequence and store their token IDs
        self.stop_token_ids = [[842, 28705], [13]]
        if stop_sequences:
            self.stop_token_ids = [tokenizer.encode(seq, add_special_tokens=False) for seq in stop_sequences]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        # Initialize a tensor to keep track of which sequences should stop
        stop_mask = torch.zeros(input_ids.shape[0], dtype=torch.bool, device=input_ids.device)
        
        # Check each stop sequence against the end of the input_ids for each sequence in the batch
        for stop_ids in self.stop_token_ids:
            if input_ids.shape[1] >= len(stop_ids):  # Ensure input_ids is long enough
                # Check if the last tokens of input_ids match the stop sequence
                match = torch.eq(input_ids[:, -len(stop_ids):], torch.tensor(stop_ids, device=input_ids.device)).all(dim=1)
                stop_mask |= match  # Update the stop mask
                if match.sum() > 0:                    
                    print(f"Mask for {stop_ids}: ", stop_mask)
        
        return stop_mask

sentence_sampling_criteria = StoppingCriteriaList([BatchSentenceStoppingCriteria(tokenizer)])

In [42]:
outputs = model.generate(
            input_tensor.to(model.device),
            # min_new_tokens=min_new_tokens,
            max_new_tokens=1000,
            return_dict_in_generate=True,
            output_scores=True,
            do_sample=True,
            temperature=0.7,
            top_k=40,
            stopping_criteria=sentence_sampling_criteria
        )
outputs.sequences.shape

Mask for [13]:  tensor([ True, False], device='mps:0')
Mask for [13]:  tensor([False,  True], device='mps:0')


torch.Size([2, 118])

In [43]:
tokenizer.batch_decode(outputs.sequences)

["<s> [INST] Q: Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?\nA: Let's think step by step. [/INST] 1. Julie has read 12 pages so far.\n</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>",
 "<s> [INST] Q: Julie is reading a 120-page book. Yesterday, she was able to read 12 pages and today, she read twice as many pages as yesterday. If she wants to read half of the remaining pages tomorrow, how many pages should she read?\nA: Let's think step by step. [/INST] 1. Julie started with 120 pages and read 12 pages yesterday, leaving her with 108 pages (120 - 12 = 108).\n"]

In [49]:
for id in outputs.sequences[0][-50:-30]:
    print("id: ", id.item(), " -- tok: ", tokenizer.decode(id))

id:  3707  -- tok:  step
id:  486  -- tok:  by
id:  3707  -- tok:  step
id:  28723  -- tok:  .
id:  733  -- tok:  [
id:  28748  -- tok:  /
id:  16289  -- tok:  INST
id:  28793  -- tok:  ]
id:  28705  -- tok:  
id:  28740  -- tok:  1
id:  28723  -- tok:  .
id:  22706  -- tok:  Julie
id:  659  -- tok:  has
id:  1220  -- tok:  read
id:  28705  -- tok:  
id:  28740  -- tok:  1
id:  28750  -- tok:  2
id:  6718  -- tok:  pages
id:  579  -- tok:  so
id:  2082  -- tok:  far


In [35]:
outputs.sequences.shape[0]

2

In [40]:
def convert_to_left_padded(tensor):
    trimmed_sequences = []
    for seq in tensor:
        # Find indices of the first and last non-pad tokens
        non_pad_indices = (seq != tokenizer.pad_token_id).nonzero(as_tuple=True)[0]
        if len(non_pad_indices) > 0:
            start_index = non_pad_indices[0]
            end_index = non_pad_indices[-1] + 1
            trimmed_sequences.append(seq[start_index:end_index])
        else:
            # if entire sequence is padding, use an empty sequence
            trimmed_sequences.append(torch.tensor([], dtype=torch.long, device=model.device))

    # Determine the maximum length after trimming
    max_length = max(len(seq) for seq in trimmed_sequences)
    padded_tensor = torch.full((tensor.shape[0], max_length), fill_value=tokenizer.pad_token_id, dtype=torch.long, device=model.device)

    # put left padding
    for i, seq in enumerate(trimmed_sequences):
        padded_tensor[i, -len(seq):] = seq

    return padded_tensor

padded_output = convert_to_left_padded(outputs.sequences)

# from utils import plot_tensor
import matplotlib.pyplot as plt
def plot_tensor(tensor):
    plt.imshow(tensor.cpu().numpy(), cmap='viridis')
    plt.axis('off')
    plt.show()

plot_tensor(padded_output)

</model.sample>